In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import glob
import shutil
import csv

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Objectif
L'objectif de ce code est de parvenir à classer nos images (crées dans le fichier COPD-notebook.ipynb) en utilisant un réseau de neurones convolutif. Pour se faire, on commence par préparer nos données en divisant nos données en données d'entraînement, de validation et de test
## Préparation des données

In [2]:
def split_data(path_to_data, path_to_save_train, path_to_save_val, split_size=0.4) :
    folders = os.listdir(path_to_data) # la liste des dossiers disponible au chemin donné
    for folder in folders :
        full_path = os.path.join(path_to_data, folder) # pour avoir le chemin complet en ajoutant le nom des dossiers
        images_paths = glob.glob(os.path.join(full_path, '*.jpeg')) # ca prend tous les fichiers à l'intérieur du dossier et les télécharge (le join il va a chaque fois ajouter le path du dossier et ajoutant le nom du fichier) ca nous retourne une liste d'images
        x_train, x_val = train_test_split(images_paths, test_size=split_size) # split en train et validation

        for x in x_train : 
            path_to_folder = os.path.join(path_to_save_train, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)
        
        for x in x_val : 
            path_to_folder = os.path.join(path_to_save_val, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)

Séparation : données d'entrainement / données de validation (60% - 40%)

In [3]:
path_to_data = "./images"
path_to_save_train = "./Training"
path_to_save_val = "./Validation"
path_to_save_test = "./Test"

In [4]:
split_data(path_to_data, path_to_save_train, path_to_save_val)

Séparation : données de validation / données de test (50% - 50%)

In [5]:
def split_val_test(path_to_data, path_to_save_test, split_size=0.5) :
    folders = os.listdir(path_to_data) # la liste des dossiers disponible au chemin donné
    for folder in folders :
        full_path = os.path.join(path_to_data, folder) # pour avoir le chemin complet en ajoutant le nom des dossiers
        images_paths = glob.glob(os.path.join(full_path, '*.jpeg')) # ca prend tous les fichiers à l'intérieur du dossier et les télécharge (le join il va a chaque fois ajouter le path du dossier et ajoutant le nom du fichier) ca nous retourne une liste d'images
        x_train, x_val = train_test_split(images_paths, test_size=split_size) # split en train et validation
        
        for x in x_val : 
            path_to_folder = os.path.join(path_to_save_test, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.move(x, path_to_folder)

In [6]:
split_val_test(path_to_save_val, path_to_save_test)

## Classification

In [7]:
def streetsigns_model(nbr_classes) :
    my_input = Input(shape=(200,160, 1))
    x= Conv2D(32, (3,3), activation='relu')(my_input)
    x= Conv2D(64, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= Conv2D(128, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= GlobalAvgPool2D()(x)
    x= Dense(64, activation='relu')(x)
    x= Dense(nbr_classes, activation='softmax')(x)
    model = Model(inputs=my_input, outputs=x)
    return model

In [8]:
model = streetsigns_model(43)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 160, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 198, 158, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 196, 156, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 98, 78, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 98, 78, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 76, 128)       73856 

## Générateur des données
Etant donné qu'on a bien mis nos données dans des dossiers différents et on a séparés convenablement entre entraînement, validation et test, on utilise un générateur de données qui va nous permettre de labeliser automatiquement ces données

In [11]:
def create_generators(batch_size, train_data_path, val_data_path, test_data_path) :
    preprocessor = ImageDataGenerator(
        rescale = 1/255. # pour assurer une division flottante
    )

    train_generator = preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        color_mode = 'grayscale', # type d'images
        shuffle = True,
        batch_size=batch_size
    )

    val_generator = preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        color_mode = 'grayscale', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    test_generator = preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        color_mode = 'grayscale', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    return train_generator, val_generator, test_generator

In [12]:
train_data_path = "./Training"
val_data_path = "./Validation"
test_data_path = "./Test"
batch_size = 4

train_generator, val_generator, test_generator= create_generators(batch_size, train_data_path, val_data_path, test_data_path)

nbr_classes = train_generator.num_classes

Found 46 images belonging to 4 classes.
Found 16 images belonging to 4 classes.
Found 16 images belonging to 4 classes.


In [13]:
model = streetsigns_model(nbr_classes)

On procède maintenant au fitting du modèle en s'assurant de sauvegarder le meilleur des modèles

In [18]:
# callbacks
path_to_save_model = './Models'
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor='accuracy', # sur quoi on se base pour voir le meilleur
    mode = 'max', # max de l'accuracy sur la validation
    save_best_only = True,
    save_freq='epoch', # ne voit qu'à la fin de l'époque
    verbose=1
) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # on choisit categorical_crossentropy car dans les générateurs on a défini categorical comme class_mode

epochs = 15

In [19]:
model.fit(
    train_generator,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = val_generator,
    callbacks=[ckpt_saver]
)

Epoch 1/15
12/12 [==============================] - ETA: 0s - loss: 0.3584 - accuracy: 0.8913
Epoch 1: accuracy improved from -inf to 0.89130, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
12/12 [==============================] - 16s 1s/step - loss: 0.3584 - accuracy: 0.8913 - val_loss: 2.6197 - val_accuracy: 0.5000
Epoch 2/15
12/12 [==============================] - ETA: 0s - loss: 0.2674 - accuracy: 0.9130
Epoch 2: accuracy improved from 0.89130 to 0.91304, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
12/12 [==============================] - 11s 945ms/step - loss: 0.2674 - accuracy: 0.9130 - val_loss: 2.7239 - val_accuracy: 0.5000
Epoch 3/15
12/12 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 0.8478
Epoch 3: accuracy did not improve from 0.91304
12/12 [==============================] - 11s 915ms/step - loss: 0.2936 - accuracy: 0.8478 - val_loss: 2.4601 - val_accuracy: 0.5000
Epoch 4/15
12/12 [=======

In [22]:
model.evaluate(test_generator)

4/4 [==============================] - 1s 248ms/step - loss: 2.1549 - accuracy: 0.5000


[2.154945135116577, 0.5]